# 🏀 NBA Predictor - Cloud Training (A100 Optimized)

## Features:
- ✅ Full historical data: **1974-2025** (50 years, ALL NBA eras)
- ✅ **10-30x faster momentum features** (vectorized NumPy)
- ✅ Neural hybrid: TabNet + LightGBM for player models
- ✅ Basketball Reference priors: ~68 advanced stats
- ✅ **Optional:** Neural hybrid for game models (+1-2% accuracy)

## Quick Start:
1. **Run the first cell** - It will prompt for file upload if needed
2. Upload 2 files: PlayerStatistics.csv.zip (41 MB) + priors_data.zip (4.8 MB)
3. Wait 18-25 minutes (A100) or 25-35 minutes (T4/L4)
4. Download models

**GPU Recommended:** A100 (fastest), L4, or T4

## What's New:
- 🚀 **Momentum features 10-30x faster** (10-20 min → 30-90 sec)
- 🧠 **Optional --game-neural flag** for TabNet game models
- 🛡️ **Overfitting prevention** for smaller game dataset

In [ ]:
# ============================================================
# SETUP & TRAIN (ALL-IN-ONE)
# ============================================================

print("📦 Installing packages...")
!pip install -q nba-api kagglehub pytorch-tabnet lightgbm scikit-learn pandas numpy tqdm

print("\n📥 Downloading code...")
import os
import shutil
from google.colab import files

os.chdir('/content')

# Remove old code if exists
if os.path.exists('meep'):
    shutil.rmtree('meep')
    print("🧹 Cleaned up old code")

!git clone https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')

print("\n📍 Code version:")
!git log -1 --oneline

# CHECK IF FILES ALREADY UPLOADED
files_exist = os.path.exists('/content/PlayerStatistics.csv') and os.path.exists('/content/priors_data')

if not files_exist:
    print("\n" + "="*70)
    print("📤 UPLOAD REQUIRED: Please upload your data files")
    print("="*70)
    print("\nYou need 2 files:")
    print("  1. PlayerStatistics.csv.zip (41 MB)")
    print("  2. priors_data.zip (4.8 MB)")
    print("\nUploading...")
    
    os.chdir('/content')
    uploaded = files.upload()
    
    # Extract files
    print("\n📦 Extracting files...")
    if os.path.exists('PlayerStatistics.csv.zip'):
        !unzip -q PlayerStatistics.csv.zip
        !rm PlayerStatistics.csv.zip
        print("✅ PlayerStatistics.csv extracted")
    
    if os.path.exists('priors_data.zip'):
        !unzip -q priors_data.zip
        print("✅ priors_data extracted")
    
    os.chdir('/content/meep')
else:
    print("\n✓ Files already uploaded, skipping upload step")

# VERIFY FILES EXIST
print("\n🔍 Pre-flight check:")
if os.path.exists('/content/PlayerStatistics.csv'):
    size_mb = os.path.getsize('/content/PlayerStatistics.csv') / 1024 / 1024
    print(f"   ✅ PlayerStatistics.csv ({size_mb:.1f} MB)")
else:
    raise FileNotFoundError("❌ PlayerStatistics.csv not found after upload!")

if os.path.exists('/content/priors_data'):
    csv_files = [f for f in os.listdir('/content/priors_data') if f.endswith('.csv')]
    print(f"   ✅ priors_data ({len(csv_files)} CSV files)")
else:
    raise FileNotFoundError("❌ priors_data not found after upload!")

# Check GPU
import torch
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'
print(f"\n🎮 GPU: {gpu_name}")

print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print("\n📊 Dataset: 1974-2025 (50+ years, 7 NBA eras)")
print("🧠 Features: Temporal + Basketball Reference priors + Momentum")
print("⚡ Neural hybrid: TabNet + LightGBM on GPU")
print("🚀 Optimizations: 10-30x faster momentum features")

if 'A100' in gpu_name:
    print("⏱️  Expected time: 18-25 minutes (A100 detected!)")
else:
    print("⏱️  Expected time: 25-35 minutes")

print("\n💡 TIP: To enable neural hybrid for game models, add --game-neural flag")
print("   (increases accuracy +1-2% but adds ~5 min training time)\n")

# RECOMMENDED: Safe mode (no --game-neural)
# Use this for your first A100 run
!python3 train_auto.py \
    --priors-dataset /content/priors_data \
    --player-csv /content/PlayerStatistics.csv \
    --verbose \
    --fresh \
    --neural-device gpu \
    --neural-epochs 50 \
    --no-window-ensemble \
    --game-season-cutoff 1974 \
    --player-season-cutoff 1974

# EXPERIMENTAL: With game neural hybrid (uncomment to test)
# !python3 train_auto.py \
#     --game-neural \
#     --priors-dataset /content/priors_data \
#     --player-csv /content/PlayerStatistics.csv \
#     --verbose \
#     --fresh \
#     --neural-device gpu \
#     --neural-epochs 50 \
#     --no-window-ensemble \
#     --game-season-cutoff 1974 \
#     --player-season-cutoff 1974

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print("\nNext: Run the Download Models cell to get your trained models")

---

**Note:** If upload fails or you need to re-upload files:
1. Delete `/content/PlayerStatistics.csv` and `/content/priors_data`
2. Re-run this cell - it will prompt for upload again

In [ ]:
# ============================================================
# STEP 2: Download Models
# ============================================================

from google.colab import files

print("📦 Packaging models...")
!zip -q -r nba_models_trained.zip models/ model_cache/

print("💾 Downloading...")
files.download('nba_models_trained.zip')

print("\n✅ Done! Extract nba_models_trained.zip to your local nba_predictor folder.")

---

## ❓ Troubleshooting

### "Loaded 0 player-games for window"
- Make sure you uploaded **PlayerStatistics.csv.zip** (39.5 MB compressed)
- File must be the ZIPPED version (not uncompressed CSV)
- Verify extraction completed successfully

### "No GPU available"
- Runtime → Change runtime type → GPU
- Select A100 (fastest), L4, or T4
- A100: 18-25 min | L4: 25 min | T4: 35 min

### "Out of memory"
- Runtime → Restart runtime
- Re-run all cells from Step 1
- Consider reducing `--neural-epochs` to 30

### "Session timeout"
- Colab Free: 12-hour limit, may disconnect
- Colab Pro: More stable for 30+ min training
- Keep browser tab active during training

---

## 📊 Dataset Details

**PlayerStatistics.csv** (Kaggle: eoinamoore/historical-nba-data-and-player-box-scores)
- **Date Range:** November 26, 1946 → November 4, 2025
- **Total Records:** 1,632,909 player-game statistics
- **Seasons:** 80 complete seasons (1947-2026)
- **Unique Dates:** 34,108 game dates

**Era Distribution:**
- Pre-3pt (≤1979): 17.8% | Early 3pt (1980-1983): 4.8%
- Hand-check (1984-2003): 30.4% | Pace Slow (2004-2012): 18.3%
- 3pt Revolution (2013-2016): 9.0% | Small Ball (2017-2020): 8.1%
- Modern (2021+): 11.6%

**priors_data.zip** (Basketball Reference statistical priors)
- Team priors: Offensive/Defensive ratings, Pace, SRS
- Player priors: Per 100 poss, Advanced stats, Shooting, Play-by-play
- ~68 advanced features from historical seasons

---

## 🎯 What's Included

**Game Models:**
- Moneyline classifier (P(home wins), isotonic calibration)
- Spread regressor (expected margin, cover probabilities)
- **Optional:** TabNet neural hybrid (--game-neural flag)

**Player Models:**
- Minutes, Points, Rebounds, Assists, 3-Pointers Made
- Team context, opponent matchup, rolling trends
- TabNet + LightGBM hybrid architecture

**Features:**
- **Momentum:** Short/medium/long-term trends (10-30x faster!)
- **Temporal:** Era categories, time-weighted samples
- **Basketball Reference:** 68 advanced priors
- **Four Factors:** eFG%, TOV%, ORB%, FTR

**Performance Optimizations (NEW):**
- Vectorized NumPy momentum calculations
- Precomputed linear regression coefficients  
- Eliminated nested Python loops
- 10-30x speedup on feature engineering

**Expected Training Output:**
```
🧠 Training on full historical dataset (1974-2025)
  • Game models: 62,085 games
  • Player models: 1.6M+ player-games
  • Features: 229 (including 68 priors)
  • Momentum features: ~90 seconds (was 10-20 min!)
  • TabNet training: 10-15 min on A100
```

---

## 🧪 Experimental: Neural Hybrid for Game Models

**What it does:**
- Adds TabNet neural network to game predictions
- Ensemble: 40% TabNet + 60% LightGBM
- Shallow architecture (3 steps) to prevent overfitting
- Strong regularization (weight decay, sparsity)

**Expected benefit:**
- +1-2% accuracy boost (62.6% → 63.5-64.5%)
- Better at capturing non-linear feature interactions

**Trade-offs:**
- +5 min training time
- More complex model (harder to debug)
- Untested in production

**How to enable:**
Uncomment the second training command in the cell above, or add `--game-neural` flag.

**Recommendation:** 
Test WITHOUT --game-neural first. If game model accuracy is good, you can skip it. Each 1% accuracy gain is significant for betting (52.4%+ beats the vig).

---

## 💡 Optional: NBA API for Live Predictions

After training, use `nba_api` for real-time game predictions:

```python
# Install: pip install nba-api
from nba_api.stats.endpoints import ScoreboardV2
from datetime import datetime

# Get today's games
today = datetime.now().strftime('%Y-%m-%d')
scoreboard = ScoreboardV2(game_date=today)
games = scoreboard.get_data_frames()[0]

# Use trained models to predict
# (requires loading models and feature engineering pipeline)
```

**Note:** NBA API is for **live predictions only**, not training (too slow, rate-limited).

---

## 📈 Version History

**v3.1 (2025-11-06)** - Performance & Neural Enhancements
- 🚀 10-30x faster momentum features (vectorized NumPy)
- 🧠 Optional neural hybrid for game models (--game-neural)
- 🛡️ Overfitting prevention (shallow TabNet, strong regularization)
- 🎯 A100 GPU support (18-25 min training)

**v3.0** - Temporal Features, Full Historical Coverage
- 1974-2025 dataset (50 years)
- Era-aware training
- Basketball Reference priors integration

**Expected Accuracy:**
- Game models: 62.6% (LightGBM) or 63.5-64.5% (with --game-neural)
- Player models: RMSE < 3.5 for points/rebounds/assists
- At 63.5%, you beat Vegas vig consistently